# 🏀 QEPC Daily Player Props

Generate player prop predictions for today's games.

## How to Use
1. Edit the games in Cell 3
2. Run all cells
3. Review predictions and edges

In [ ]:
# Setup
import sys
from pathlib import Path

for folder in [Path.cwd(), Path.cwd() / "props", Path.cwd().parent]:
    if (folder / "player_props_engine.py").exists():
        sys.path.insert(0, str(folder))
        break

import pandas as pd
from player_props_engine import PlayerPropsEngine

# Load engine
engine = PlayerPropsEngine("data/raw/PlayerStatistics.csv")
engine.load_data()
print("✅ Engine loaded!")

In [ ]:
# Define Today's Games
# Format: (team, opponent, is_home)

TODAYS_GAMES = [
    ("Los Angeles Lakers", "Golden State Warriors", True),
    ("Boston Celtics", "Miami Heat", False),
    # Add more games here...
]

# Players to predict (leave empty for auto-detect from team)
SPECIFIC_PLAYERS = [
    # "LeBron James",
    # "Stephen Curry",
]

# Props to predict
PROPS = ['PTS', 'REB', 'AST', 'PRA']

print(f"Games: {len(TODAYS_GAMES)}")
print(f"Props: {PROPS}")

In [ ]:
# Generate Predictions
all_predictions = []

for team, opponent, is_home in TODAYS_GAMES:
    print(f"\n{'='*50}")
    print(f"🏀 {team} vs {opponent} ({'Home' if is_home else 'Away'})")
    print("="*50)
    
    # Get team projections
    df = engine.get_team_projections(
        team=team,
        opponent=opponent, 
        is_home=is_home,
        props=PROPS,
        min_minutes=20.0
    )
    
    if df.empty:
        print("  No predictions available")
        continue
    
    # Display
    for prop in PROPS:
        prop_df = df[df['prop_type'] == prop].head(5)
        if not prop_df.empty:
            print(f"\n{prop}:")
            for _, row in prop_df.iterrows():
                print(f"  {row['player_name']:20} | {row['projection']:5.1f} | {row['confidence']}")
    
    all_predictions.append(df)

if all_predictions:
    full_df = pd.concat(all_predictions, ignore_index=True)
    print(f"\n\n📊 Total predictions: {len(full_df)}")

In [ ]:
# Edge Finder
# Enter betting lines here to find edges

BETTING_LINES = {
    "LeBron James": {"PTS": 25.5, "REB": 7.5, "AST": 8.5},
    "Stephen Curry": {"PTS": 27.5, "3PM": 4.5},
    # Add more players/lines...
}

if BETTING_LINES:
    print("\n" + "="*60)
    print("💰 EDGE ANALYSIS")
    print("="*60)
    
    edges_df = engine.find_edges(BETTING_LINES, min_edge=0.03)
    
    if not edges_df.empty:
        print("\n🎯 Opportunities Found:")
        display(edges_df)
    else:
        print("No edges >= 3% found")
else:
    print("Add betting lines to BETTING_LINES to find edges")

In [ ]:
# Single Player Deep Dive
PLAYER_TO_ANALYZE = "LeBron James"  # Change this
OPPONENT = "Warriors"

print(f"\n{'='*60}")
print(f"📊 DEEP DIVE: {PLAYER_TO_ANALYZE}")
print("="*60)

for prop in ['PTS', 'REB', 'AST', '3PM', 'PRA']:
    pred = engine.predict(PLAYER_TO_ANALYZE, prop, opponent=OPPONENT)
    
    if pred:
        print(f"\n{prop}:")
        print(f"  Projection: {pred.projection}")
        print(f"  Range: {pred.floor} - {pred.ceiling}")
        print(f"  Std Dev: {pred.std_dev}")
        print(f"  Confidence: {pred.confidence}")
        print(f"  Games Analyzed: {pred.games_analyzed}")
        
        # Show over/under for common lines
        if prop == 'PTS':
            for line in [20.5, 22.5, 25.5, 27.5, 30.5]:
                print(f"    Over {line}: {pred.over_prob(line):.1%}")